In [ ]:
from google.colab import drive
from pathlib import Path

drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import nltk
wn = nltk.WordNetLemmatizer()
ps = nltk.PorterStemmer()
import glob
import os
import re
import shutil
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
import csv
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
import string
import itertools
import zipfile
import json
punctuation = string.punctuation
stopwordsset = set(stopwords.words("english"))
stopwordsset.add('rt')
stopwordsset.add("'s")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


Clean up our twitter data and gives us a semantic network. We want to figure out what words are the most related. 

Stemming gets us to the root of the word. 

You also want to remove stopwods/common words. 

Lemmitization is taking the root of the word. 

In [ ]:
#Removing urls
def removeURL(text):
  result = re.sub(r"http\S+", "", text)
  return result
#Extracting contextual words from a sentence
# tokenizing is taking out all the words in a sentence and turning it into tokens/words
def tokenize(text):
  #lower case
  text = text.lower()
  #split into individual words
  words = word_tokenize(text)
  return words
#stem - peaches : peach : reduce the number of repeated words
def stem(tokenizedtext):
  rootwords = []
  for aword in tokenizedtext:
    aword = ps.stem(aword)
    rootwords.append(aword)
  return rootwords
#removes useless words such as a, an, the
def stopWords(tokenizedtext):
  goodwords = []
  for aword in tokenizedtext:
    if aword not in stopwordsset:
      goodwords.append(aword)
  return goodwords
# feature reduction. taking words and getting their roots and graphing only the root words
def lemmatizer(tokenizedtext):
  lemmawords = []
  for aword in tokenizedtext:
    aword = wn.lemmatize(aword)
    lemmawords.append(aword)
  return lemmawords
#inputs a list of tokens and returns a list of unpunctuated tokens/words
def removePunctuation(tokenizedtext):
  nopunctwords = []
  for aword in tokenizedtext:
    if aword not in punctuation:
      nopunctwords.append(aword)
  cleanedwords = []
  for aword in nopunctwords:
    aword = aword.translate(str.maketrans('', '', string.punctuation))
    cleanedwords.append(aword)
  return cleanedwords

In [ ]:
tweetzipfiles = glob.glob('/content/drive/My Drive/Colab Notebooks/ERPvendors/Main Folder/*.zip')
tweetzipfiles

['/content/drive/My Drive/Colab Notebooks/ERPvendors/Main Folder/@Oracle.zip',
 '/content/drive/My Drive/Colab Notebooks/ERPvendors/Main Folder/@salesforce.zip',
 '/content/drive/My Drive/Colab Notebooks/ERPvendors/Main Folder/@SAP.zip']

Using zipfile to open the path, and make it an actual file object. We are loading in the json object. We need to extract out the text from the tweet.



In [ ]:
from time import sleep
uniquewords = {}
count = 0

for tweetzipfile in tweetzipfiles:
  zf = zipfile.ZipFile(tweetzipfile)
  for i, obj in enumerate(zf.infolist()):
    tweetjson = json.load(zf.open(obj))
    count += 1
    if count % 1000 == 0:
      print(count)
    
    text = tweetjson['text']
    #natural language processing: clean the tweet
    nourlstext = removeURL(text)
    tokenizedtext = tokenize(nourlstext)
    nostopwordstext = stopWords(tokenizedtext)
    lemmatizedtext = lemmatizer(nostopwordstext)
    nopuncttext = removePunctuation(lemmatizedtext)
    
  
    for aword in nopuncttext:
      if aword in uniquewords:
        uniquewords[aword] += 1
      if aword not in uniquewords:
        uniquewords[aword] = 1

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000


The dictionary can be used to figure out what words you want to use. 

Helps you narrow down the size of your vocab. 

In [ ]:
len(uniquewords)

17729

These are the words to include in a semantic network analysis.

In [ ]:
#list of unique words: key-word = number of times the word appears
wordstoinclude = set()
wordcount = 0
for aword in uniquewords:
  if uniquewords[aword] > 25:
    wordcount += 1
    wordstoinclude.add(aword)

In [ ]:
print (wordcount)

1383


We want gephi to know that this is an undirected network. 

In [ ]:
edgelist1 = open('/content/drive/My Drive/Colab Notebooks/ERPvendors/Main Folder/gephi-semantic.csv','w')
csvwriter = csv.writer(edgelist1)

header = ['Source','Target','Type']
csvwriter.writerow(header) #write first row with header

print ('Writing Edge List')

uniquewords = {}
count = 0

for tweetzipfile in tweetzipfiles:
  zf = zipfile.ZipFile(tweetzipfile)
  for i, obj in enumerate(zf.infolist()):
    tweetjson = json.load(zf.open(obj))
    count += 1
    if count % 1000 == 0:
      print(count)

    text = tweetjson['text']

    nourlstext = removeURL(text)
    tokenizedtext = tokenize(nourlstext)
    nostopwordstext = stopWords(tokenizedtext)
    lemmatizedtext = lemmatizer(nostopwordstext)
    nopuncttext = removePunctuation(lemmatizedtext)

    goodwords = []
    for aword in nopuncttext:
      if aword in wordstoinclude:
        goodwords.append(aword.replace(',',''))
    
    allcombos = itertools.combinations(goodwords,2)
    for acombo in allcombos:
        row = []
        for anode in acombo:
            row.append(anode)
        row.append('Undirected')
        csvwriter.writerow(row)

edgelist1.close()


Writing Edge List
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000


In [ ]:
uniquewords = {}
count = 0

for tweetzipfile in tweetzipfiles:
  zf = zipfile.ZipFile(tweetzipfile)
  for i, obj in enumerate(zf.infolist()):
    tweetjson = json.load(zf.open(obj))
    count += 1
    if count % 1000 == 0:
      print(count)

    text = tweetjson['text']

    nourlstext = removeURL(text)
    tokenizedtext = tokenize(nourlstext)
    nostopwordstext = stopWords(tokenizedtext)
    lemmatizedtext = lemmatizer(nostopwordstext)
    nopuncttext = removePunctuation(lemmatizedtext)

    goodwords = []
    for aword in nopuncttext:
      if aword in wordstoinclude:
        goodwords.append(aword.replace(',',''))
    
    allcombos = itertools.combinations(goodwords,2)
    for acombo in allcombos:
        row = []
        for anode in acombo:
            row.append(anode)
        row.append('Undirected')
        csvwriter.writerow(row)

ValueError: ignored